<a href="https://colab.research.google.com/github/Jesmeeksingh/Machine_Learning_projects/blob/main/Hinglish_whatsapp_chat_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
with open('thefreakschat.txt','r') as file:
  hinglish_text = file.read()

In [81]:
# hinglish_text

In [3]:
lines = hinglish_text.split('\n')

In [82]:
messages = []
keywords_to_exclude = ['<Media omitted>','null']
for line in lines:
    message_index = line.find('-') + 2
    message = line[message_index:]
    message_index = message.find(':')
    if message_index != -1:  # Check if ':' is found
        message = message[message_index + 1:]  # Skip the ':'
        if not any(keyword in message for keyword in keywords_to_exclude):
          messages.append(message.strip())


modified_data = '\n'.join(messages)
# modified_data

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts([modified_data])

In [83]:
# tokenizer.word_index

In [9]:
input_sequences = []
for sentence in modified_data.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
     input_sequences.append(tokenized_sentence[:i+1])

In [84]:
# input_sequences

In [11]:
max_len = max([len(x) for x in input_sequences])
max_len

118

In [12]:
#zero padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
padded_input_sequences = pad_sequences(input_sequences,maxlen = max_len,padding = 'pre')

In [14]:
X = padded_input_sequences[:,:-1]

In [85]:
# X

In [16]:
X.shape

(90815, 117)

In [17]:
Y = padded_input_sequences[:,-1]

In [86]:
# Y

In [18]:
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y,num_classes=12114)

In [87]:
# Y

In [20]:
Y.shape

(90815, 12114)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [22]:
model = Sequential()
model.add(Embedding(12114,100,input_length=117))
model.add(LSTM(150))
model.add(Dense(12114,activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 117, 100)          1211400   
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 12114)             1829214   
                                                                 
Total params: 3191214 (12.17 MB)
Trainable params: 3191214 (12.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.fit(X,Y,epochs=30)

Epoch 1/30
2838/2838 [==============================] - 64s 21ms/step - loss: 7.3314 - accuracy: 0.0444
Epoch 2/30
2838/2838 [==============================] - 40s 14ms/step - loss: 6.6229 - accuracy: 0.0809
Epoch 3/30
2838/2838 [==============================] - 39s 14ms/step - loss: 6.1136 - accuracy: 0.1099
Epoch 4/30
2838/2838 [==============================] - 38s 14ms/step - loss: 5.6572 - accuracy: 0.1321
Epoch 5/30
2838/2838 [==============================] - 39s 14ms/step - loss: 5.2269 - accuracy: 0.1548
Epoch 6/30
2838/2838 [==============================] - 38s 13ms/step - loss: 4.8137 - accuracy: 0.1827
Epoch 7/30
2838/2838 [==============================] - 38s 13ms/step - loss: 4.4283 - accuracy: 0.2165
Epoch 8/30
2838/2838 [==============================] - 38s 14ms/step - loss: 4.0716 - accuracy: 0.2583
Epoch 9/30
2838/2838 [==============================] - 37s 13ms/step - loss: 3.7514 - accuracy: 0.3027
Epoch 10/30
2838/2838 [==============================] - 37s 13m

In [80]:
import numpy as np
text = "tanvir ko"
for i in range(5):

  token_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([token_text],maxlen = 117,padding='pre')
  pos = np.argmax(model.predict(padded_token_text))
  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)


1/1 [==============================] - 0s 20ms/step
tanvir ko bhi
1/1 [==============================] - 0s 22ms/step
tanvir ko bhi utha
1/1 [==============================] - 0s 19ms/step
tanvir ko bhi utha lenge
1/1 [==============================] - 0s 21ms/step
tanvir ko bhi utha lenge ab
1/1 [==============================] - 0s 19ms/step
tanvir ko bhi utha lenge ab ab
